<a href="https://colab.research.google.com/github/zhangxs131/garbage/blob/main/%E5%85%B3%E9%94%AE%E8%AF%8D%E6%8A%BD%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#装载谷歌云盘
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#查看数据
!head -n 10 drive/MyDrive/NLPProject/data/cnews.train.txt

体育	马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有摆脱雨水的困扰。7月31日下午6点，国奥队的日常训练再度受到大雨的干扰，无奈之下队员们只慢跑了25分钟就草草收场。31日上午10点，国奥队在奥体中心外场训练的时候，天就是阴沉沉的，气象预报显示当天下午沈阳就有大雨，但幸好队伍上午的训练并没有受到任何干扰。下午6点，当球队抵达训练场时，大雨已经下了几个小时，而且丝毫没有停下来的意思。抱着试一试的态度，球队开始了当天下午的例行训练，25分钟过去了，天气没有任何转好的迹象，为了保护球员们，国奥队决定中止当天的训练，全队立即返回酒店。在雨中训练对足球队来说并不是什么稀罕事，但在奥运会即将开始之前，全队变得“娇贵”了。在沈阳最后一周的训练，国奥队首先要保证现有的球员不再出现意外的伤病情况以免影响正式比赛，因此这一阶段控制训练受伤、控制感冒等疾病的出现被队伍放在了相当重要的位置。而抵达沈阳之后，中后卫冯萧霆就一直没有训练，冯萧霆是7月27日在长春患上了感冒，因此也没有参加29日跟塞尔维亚的热身赛。队伍介绍说，冯萧霆并没有出现发烧症状，但为了安全起见，这两天还是让他静养休息，等感冒彻底好了之后再恢复训练。由于有了冯萧霆这个例子，因此国奥队对雨中训练就显得特别谨慎，主要是担心球员们受凉而引发感冒，造成非战斗减员。而女足队员马晓旭在热身赛中受伤导致无缘奥运的前科，也让在沈阳的国奥队现在格外警惕，“训练中不断嘱咐队员们要注意动作，我们可不能再出这样的事情了。”一位工作人员表示。从长春到沈阳，雨水一路伴随着国奥队，“也邪了，我们走到哪儿雨就下到哪儿，在长春几次训练都被大雨给搅和了，没想到来沈阳又碰到这种事情。”一位国奥球员也对雨水的“青睐”有些不解。
体育	商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也来了。距离首战72小时当口，中国女足彻底从“恐瑞症”当中获得解脱，因为商瑞华已经找到了瑞典人的软肋。找到软肋，保密4月20日奥运会分组抽签结果出来后，中国姑娘就把瑞典锁定为关乎奥运成败的头号劲敌，因为除了浦玮等个别老将之外，现役女足将士竟然没有人尝过击败瑞典的滋味。在中瑞两队共计15次交锋的历史上，中国队6胜3平6负与瑞典队平分秋色，但从2001年起至今近8年时间，中国在同瑞典连续5次交锋中均未尝胜

#事件检测

首先构造一下数据集吧

In [10]:
from torch.utils.data import Dataset
import random
from sklearn.utils import shuffle

split_ratio=[0.9,0.05,0.05]

with open('drive/MyDrive/NLPProject/data/cnews.train.txt','r',encoding='utf-8',errors='ignore') as f:
      content=f.readlines()

content=shuffle(content)


def get_some_data(sum=1000):
  labels=[]
  text=[]
  for i in content[:sum]:
    t=i.split('\t')
    labels.append(t[0])
    text.append(t[1])

  return labels,text

labels,text=get_some_data(10000)
print(len(labels))
for i in range(5):
  print(labels[i],text[i])

10000
时政 国务院共出台14部与慈善相关规章福建省泉州市的王钦辉，因家境困难，手拿大学录取通知书却无钱上学。福建匹克集团的许景南与泉州慈善总会筹备成立的匹克慈善基金为这个孩子圆了大学梦。像王钦辉一样，千千万万困难群体在慈善组织的帮助下，渡过难关，享受人间温暖。改革开放30年来，我国慈善事业发展迅猛，成就斐然。发展慈善事业列入国家规划党的十六届四中全会强调要“健全社会保险、社会救助、社会福利和慈善事业相衔接的社会保障体系”，“慈善事业”第一次写进执政党的重要文献。2005年3月，国务院将“支持慈善事业发展”第一次写入政府工作报告；2006年政府工作报告、2007年政府工作报告、“十一五”规划等也提出了鼓励开展社会慈善的要求。党和政府的重视，为我国慈善事业的发展指明了前进方向，注入了强劲动力。民政部门承担起管理、指导慈善事业发展的职责。2005年11月，民政部在首届中华慈善大会上发布了《中国慈善事业发展指导纲要（2006—2010年）》，首次明确了我国慈善事业发展的目标、原则和措施，为推动慈善事业全面协调发展奠定了基础。慈善法律法规是发展慈善事业的制度保障。截至2008年10月底，全国人大及常务委员会颁布的《中华人民共和国红十字会法》、《中华人民共和国公益事业捐赠法》等6部法律中涉及到慈善相关内容。在国务院制定的行政法规中，《中华人民共和国企业所得税法实施条例》等7部行政法规中对慈善组织作出了明确规定。国务院各部门共出台《救灾捐赠管理办法》等14部与慈善相关部门规章、270余件相关规范性文件。国家还通过税收减免促进慈善事业的发展，鼓励慈善组织对所拥有资产进行保值增值，并对用于慈善公益性活动和项目的增值部分免征所得税。2008年1月1日起实行的新《企业所得税法》将企业捐赠免税额度由原来的3％提高到12％，进一步激发了企业捐赠的积极性。社会广泛关注，人民积极参与随着我国社会经济的发展，人民生活水平的不断提高，慈善事业日益受到社会各界广泛关注，而且吸引广大群众积极参与，成为社会保障体系的重要有益补充。慈善在我国社会发展进程中扮演着越来越重要的角色，尤其在1998年抗洪、2005年援助印度洋海啸灾区、2008年抵御南方低温雨雪冰冻灾害和抗击汶川特大地震灾害过程中，慈善的身影无处不在，发挥了重要作用。大量活动和项目帮助困难民众在改革开放30年里，我国慈善事业在发展中壮大，在

In [11]:
print(len(list(set(labels))))
print(list(set(labels)))

10
['家居', '游戏', '娱乐', '科技', '房产', '时尚', '财经', '教育', '时政', '体育']


In [19]:
#加载停用词
import jieba

with open('drive/MyDrive/NLPProject/data/cn_stopwords.txt','r',encoding='utf-8') as f:
  stopwords=f.readlines()


In [21]:
#使用sklearn的tf-idf和tf进行抽取关键词
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

tfidf=TfidfVectorizer(
    ngram_range=(1,1),
    stop_words=stopwords,
    tokenizer=jieba.lcut,
    max_features=1000)
tfidf.fit_transform(text)

tf=CountVectorizer(ngram_range=(1,1),stop_words=stopwords,tokenizer=jieba.lcut,max_features=500)
tf.fit_transform(text)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\n', '"', '#', '##', '$', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', '...', '......', '...................', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', ']', '_', 'a', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', 'aren', 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'bee

<10000x500 sparse matrix of type '<class 'numpy.int64'>'
	with 859206 stored elements in Compressed Sparse Row format>

In [26]:
#显示tf-idf的特征值，选择高频作为keywords
name=tfidf.get_feature_names()
feature=[i for i in name if len(i)>1]
print('tfidf features:',feature[50:80])

name2=tf.get_feature_names()
feature2=[i for i in name2 if len(i)>1]
print('tf features:',feature2[50:80])

#通过分析结果，感觉tf-idf对提取关键信息，比较依赖停用词表，最好加上POS词性限制，效果可能会更好一点。

tfidf features: ['万平方米', '三个', '三分', '上升', '上半年', '上市', '上映', '上海', '上涨', '上述', '下跌', '下降', '不仅', '不会', '不到', '不同', '不少', '不断', '不是', '不能', '不要', '不过', '不错', '专业', '专家', '世界', '世纪', '业主', '业务', '业绩']
tf features: ['交易', '产品', '人们', '人员', '人士', '人民币', '亿元', '什么', '今天', '今年', '介绍', '他们', '以上', '以及', '以来', '价格', '任何', '企业', '优势', '但是', '住房', '作为', '作品', '使用', '信息', '债券', '全国', '全球', '公司', '公布']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
#使用jieba中的函数tf-idf和textrank方法进行关键词提取
import jieba.analyse


#选择top30关键词，词性POS选择ns地名，n名词,vn 动名词，v 动词

key=jieba.analyse.extract_tags((' ').join(text),topK=30,allowPOS=('ns','n'))
print('tf-idf:',key)
key2=jieba.analyse.textrank((' ').join(text),topK=30,allowPOS=('ns','n'))
print('textrank:',key2)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.141 seconds.
Prefix dict has been built successfully.


tf-idf: ['学生', '游戏', '中国', '基金', '电影', '大学', '玩家', '市场', '时间', '北京', '记者', '银行', '美国', '问题', '项目', '球队', '国际', '篮板', '公司', '学校', '企业', '奇才', '英国', '情况', '时尚', '开发商', '时候', '火箭', '国家', '热火']
textrank: ['中国', '基金', '市场', '美国', '学生', '北京', '游戏', '公司', '国际', '大学', '时间', '银行', '项目', '问题', '记者', '企业', '电影', '经济', '国家', '英国', '上海', '情况', '学校', '日本', '方面', '部分', '时尚', '城市', '全球', '专业']


#word2vec进行关键词提取
一般都使用中文wiki语料，训练word2vec词向量，但由于语料过大，这里只给出下载地址，并且可以直接使用gensim中的处理函数进行处理。

我这里就是用自己的语料了

In [ ]:
#中文wiki语料下载，差不多压缩前2.5G，解压后8个G左右
!wget https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles-multistream.xml.bz2

In [ ]:
#处理wiki中文语料，这里处理完得到的是一篇文章一行的 繁体语料
from gensim.corpora import WikiCorpus

inp='zhwiki-latest-pages-articles-multistream.xml.bz2'
outp='wiki_zh.txt'

wiki=WikiCorpus(inp,lemmatize=False,dictionary={})

i=0
output=open(outp,'w',encoding='utf-8')
for text in wiki.get_texts():
  output.write((' ').join(text)+'\n')
  i+=1
  if i%1000==0:
    print("SAVE {} articles".format(i))
output.close()

In [ ]:
#繁体字转简体字，使用opencc
!pip install opencc-python

In [ ]:
import opencc
cc=opencc.OpenCC('t2s')
print(cc.convert(u'Open Chinese Convert（OpenCC）「開放中文轉換」，是一個致力於中文簡繁轉換的項目，提供高質量詞庫和函數庫(libopencc)。'))

In [ ]:
#这里下载识别，直接使用linux下clone opencc的包

In [30]:
!apt-get install opencc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopencc2 libopencc2-data
The following NEW packages will be installed:
  libopencc2 libopencc2-data opencc
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,390 kB of archives.
After this operation, 7,332 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencc2-data all 1.0.4-5 [106 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencc2 amd64 1.0.4-5 [1,199 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 opencc amd64 1.0.4-5 [85.3 kB]
Fetched 1,390 kB in 1s (1,750 kB/s)
Selecting previously unselected package libopencc2-data.
(Reading database ... 155320 files and directories curren

In [32]:
!echo '听说俄罗斯正在打乌克兰，不知道事情会发展的如何呀。'> zh_s.txt
!opencc -i zh_s.txt -o zh_t.txt 

#可以使用 -c zht2zhs 是将繁体字转换为简体字 

In [33]:
!cat zh_s.txt
!cat zh_t.txt

听说俄罗斯正在打乌克兰，不知道事情会发展的如何呀。
聽說俄羅斯正在打烏克蘭，不知道事情會發展的如何呀。


In [36]:
import jieba
import gensim
from gensim.corpora import Dictionary
import random

data=[jieba.lcut(i) for i in text[:100]]
sampler=text[random.randint(0,9999)]
sampler_tokenized=jieba.lcut(sampler)
dictionary=Dictionary(data)
print('dictionary',dictionary)
doc=dictionary.doc2bow(sampler_tokenized)
print(doc)

dictionary Dictionary(9876 unique tokens: ['\n', ' ', '1', '1.1', '10']...)
[(0, 1), (20, 1), (32, 2), (33, 2), (34, 4), (35, 7), (55, 1), (57, 1), (62, 2), (65, 1), (66, 1), (102, 2), (143, 1), (155, 3), (237, 1), (249, 3), (256, 1), (286, 2), (288, 1), (304, 1), (308, 1), (322, 1), (372, 14), (405, 1), (437, 2), (463, 1), (474, 1), (509, 1), (538, 1), (547, 2), (608, 1), (627, 1), (646, 1), (730, 1), (733, 1), (735, 1), (779, 1), (935, 1), (941, 1), (1025, 1), (1054, 1), (1088, 1), (1182, 1), (1225, 1), (1251, 1), (1358, 1), (1380, 1), (1436, 2), (1453, 1), (1589, 1), (1611, 1), (1725, 1), (1744, 1), (1945, 1), (1981, 1), (2234, 1), (2771, 1), (3252, 1), (3265, 1), (3427, 1), (3468, 2), (3627, 1), (3722, 1), (3932, 1), (4137, 1), (4238, 1), (4427, 1), (4501, 1), (4551, 1), (4576, 1), (4760, 1), (4928, 1), (5475, 1), (5531, 2), (5600, 1), (5606, 8), (5636, 1), (5756, 8), (6292, 1), (6932, 1), (7320, 1), (8579, 1), (8695, 1), (9217, 1), (9379, 1), (9516, 1), (9803, 2), (9810, 1), (9845

In [ ]:
#使用word2vec进行关键词提取

import jieba.posseg
import gensim

inp = 'wiki.zh.text.vector'
model = gensim.models.KeyedVectors.load_word2vec_format(inp, binary=False)
pos=['n','ns']

words=jieba.posseg.cut((' ').join(text))
cleaned_words=[]
for i in words:
  if i.word not in cleaned_words and i.word not in stopwords and i.flag in pos:
    cleaned_words.append(i.word)


result=[]
for i in cleaned_words:
  if i in model:
    result.append({'name':i.encode('utf-8'),'vec':model(i)})


print(result[:5])

  



In [12]:
#使用LDA主题模型

from gensim.models import LdaModel
from gensim.corpora import Dictionary

#先进行分词处理

data=[jieba.lcut(i) for i in text]

#将数据转换为词袋模型

dictionary=Dictionary(data)
#过滤掉一些频率高的词
dictionary.filter_n_most_frequent(200)
corpus=[dictionary.doc2bow(t) for t in data]
print(corpus[:5])

#将bow数据喂给LDAmodel

lda=LdaModel(corpus=corpus,id2word=dictionary,num_topics=10)

topic_list=lda.print_topics(20)
for i in topic_list:
  print(i)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 3), (8, 2), (9, 3), (10, 3), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 3), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 5), (35, 1), (36, 1), (37, 4), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 5), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 3), (97, 1), (98, 1), (99, 1), (100, 1), (101, 3), (102, 2), (103, 4), (104, 2), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 3)

由LDA分类结果可知:'家居', '游戏', '娱乐', '科技', '房产', '财经', '时尚', '教育', '时政', '体育'
可能是数据过少导致，LDA效果不明显

In [ ]:
#尝试使用lda进行分类

In [16]:
import random

#随机选择一个文本

tlabel,ttext=content[random.randint(0,56666)].split('\t')
print(tlabel,ttext)

tt=jieba.lcut(ttext)
doc_bow=dictionary.doc2bow(tt)
doc_lda=lda[doc_bow]
print(doc_lda)



教育 出国留学理财常见问题：留学人员海外金融消费6、对留学人员的海外金融消费，有哪些建议？不宜携带太多外币现钞出国。带够2—3个月的生活费就可以了，其余的生活费、学费可通过银行汇票或电汇的方式支付。要对海外生活费、学费的支出做到"心中有数，事先规划",兑换学费和生活费的先后顺序也有讲究。按照外管规定，如果兑换了学费再换生活费，并超过了年度总额5万美元的限定，需要提交对方学校关于这笔学费的缴费收据和在国外学校的在读证明，凭这些材料经银行核销后才能兑换所需的生活费。所以先兑换生活费再兑换学费更好。应养成刷卡消费的习惯。留学生境外消费，家长可以在国内用人民币帮助还款。本着以下原则即可：电汇票汇付学费，日常开支国际卡和信用卡、旅行支票来应急。

[(1, 0.016729794), (3, 0.21899685), (5, 0.27541983), (6, 0.4498619), (8, 0.03448644)]
